In [146]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd

spark = sphynx.get_spark(executor_count=8, app_name='pdu_syp', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 8 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: pdu_syp...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:26


In [145]:
sphynx.stop()

In [137]:
device="pc"

bp_event_name = "202108_blackpink"
bp_start_date = "2021-08-04"
bp_end_date = "2021-08-10"

pcs5_event_name = "202109_pcs5"
pcs5_start_date = "2021-09-08"
pcs5_end_date = "2021-09-14"

tg_event_name = "202109_thanksgiving"
tg_start_date = "2021-09-17"
tg_end_date = "2021-09-23"

hw_event_name = "202110_halloween"
hw_start_date = "2021-10-20"
hw_end_date = "2021-10-26"

pgc_event_name = "202111_pgc"
pgc_start_date = "2021-11-12"
pgc_end_date = "2021-11-18"

In [114]:
# bp_user = load_data_mart(start_date=bp_start_date, end_date=bp_end_date, device=device, table_name="user_master").where(col("lastlogindate") >= bp_start_date)
bp_gcoin = load_data_mart(start_date=bp_start_date, end_date=bp_end_date, device=device, table_name="gcoin_use").where(col("event_name") == bp_event_name)

In [117]:
bp_gcoin.withColumn("product_group", when(col("product_id") == "itemdesc.13000318", "MEGA BUNDLE").otherwise("BUNDLE")).groupBy("date", "product_group").agg(countDistinct("account_id").alias("pu")).orderBy("date", "product_group").toPandas()

,date,product_group,pu
0,2021-08-04,BUNDLE,22888
1,2021-08-04,MEGA BUNDLE,22570
2,2021-08-05,BUNDLE,16427
3,2021-08-05,MEGA BUNDLE,11095
4,2021-08-06,BUNDLE,11327
5,2021-08-06,MEGA BUNDLE,6441
6,2021-08-07,BUNDLE,9700
7,2021-08-07,MEGA BUNDLE,4915
8,2021-08-08,BUNDLE,7663
9,2021-08-08,MEGA BUNDLE,3496


In [118]:
pcs5_gcoin = load_data_mart(start_date=pcs5_start_date, end_date=pcs5_end_date, device=device, table_name="gcoin_use").where(col("event_name") == pcs5_event_name)

In [121]:
pcs5_gcoin.withColumn("product_group", when(col("product_id") == "itemdesc.13000328", "ULTIMATE BUNDLE").when(col("product_id").isin(["itemdesc.13000329"]), "SET").otherwise("OTHERS")).groupBy("date", "product_group").agg(countDistinct("account_id").alias("pu"), (sum(col("paid_use"))/100).alias("revenue")).orderBy("date", "product_group").toPandas()

,date,product_group,pu,revenue
0,2021-09-08,OTHERS,24815,143853.0
1,2021-09-08,SET,15614,244688.5
2,2021-09-08,ULTIMATE BUNDLE,10760,393024.2
3,2021-09-09,OTHERS,11822,73580.9
4,2021-09-09,SET,6767,104793.1
5,2021-09-09,ULTIMATE BUNDLE,4009,145287.5
6,2021-09-10,OTHERS,8618,56216.2
7,2021-09-10,SET,4184,66579.2
8,2021-09-10,ULTIMATE BUNDLE,2399,87296.6
9,2021-09-11,OTHERS,7260,48934.1


In [122]:
tg_gcoin = load_data_mart(start_date=tg_start_date, end_date=tg_end_date, device="pc", table_name="gcoin_use").where(col("event_name") == tg_event_name)

In [125]:
# bundle_list = ["itemdesc.13000354", "itemdesc.13000350", "itemdesc.13000349", "itemdesc.13000351", "itemdesc.13000352", "itemdesc.13000353"]

In [126]:
bundle_list = ["itemdesc.13000354", "itemdesc.13000350", "itemdesc.13000349", "itemdesc.13000351", "itemdesc.13000352", "itemdesc.13000353"]
tg_gcoin.withColumn("product_group", when(col("product_id") == "itemdesc.13000348", "ULTIMATE BUNDLE").when(col("product_id").isin(bundle_list), "BUNDLE").otherwise("OTHERS")).groupBy("date", "product_group").agg(countDistinct("account_id").alias("pu"), (sum("paid_use")/100).alias("revenue")).orderBy("date", "product_group").toPandas()

,date,product_group,pu,revenue
0,2021-09-17,BUNDLE,29257,473607.1
1,2021-09-17,OTHERS,23428,198959.5
2,2021-09-17,ULTIMATE BUNDLE,12720,619795.6
3,2021-09-18,BUNDLE,15145,237553.0
4,2021-09-18,OTHERS,12602,104848.9
5,2021-09-18,ULTIMATE BUNDLE,4581,226838.8
6,2021-09-19,BUNDLE,10547,162585.8
7,2021-09-19,OTHERS,9151,73284.7
8,2021-09-19,ULTIMATE BUNDLE,2743,136055.0
9,2021-09-20,BUNDLE,8035,122578.7


In [128]:
hw_gcoin = load_data_mart(start_date=hw_start_date, end_date=hw_end_date, device="pc", table_name="gcoin_use").where(col("event_name") == hw_event_name)

In [130]:
# hw_bundle_list = ["itemdesc.13000358", "itemdesc.13000360", "itemdesc.13000357", "itemdesc.13000359", "itemdesc.13000361"]

In [131]:
hw_bundle_list = ["itemdesc.13000358", "itemdesc.13000360", "itemdesc.13000357", "itemdesc.13000359", "itemdesc.13000361"]
hw_gcoin.withColumn("product_group", when(col("product_id") == "itemdesc.13000362", "ULTIMATE BUNDLE").when(col("product_id").isin(hw_bundle_list), "BUNDLE").otherwise("OTHERS")).groupBy("date", "product_group").agg(countDistinct("account_id").alias("pu"), (sum("paid_use")/100).alias("revenue")).orderBy("date", "product_group").toPandas()

,date,product_group,pu,revenue
0,2021-10-20,BUNDLE,3861,55190.35
1,2021-10-20,OTHERS,10157,44776.95
2,2021-10-20,ULTIMATE BUNDLE,1249,86677.50
3,2021-10-21,BUNDLE,1619,22941.90
4,2021-10-21,OTHERS,4689,19354.80
5,2021-10-21,ULTIMATE BUNDLE,399,27929.50
6,2021-10-22,BUNDLE,1154,16705.00
7,2021-10-22,OTHERS,3213,13058.35
8,2021-10-22,ULTIMATE BUNDLE,216,15265.50
9,2021-10-23,BUNDLE,988,14080.20


In [132]:
pgc_gcoin = load_data_mart(start_date=pgc_start_date, end_date=pgc_end_date, device="pc", table_name="gcoin_use").where(col("event_name") == pgc_event_name)

In [133]:
pgc_gcoin.select("product_name").distinct().show()

+--------------------+
|        product_name|
+--------------------+
|PGC 2021 CONTINGE...|
|PGC 2021 MEGA BUN...|
|PGC 2021 CONTINGE...|
+--------------------+



In [134]:
pgc_gcoin.withColumn("product_group", when(col("product_id") == "itemdesc.13000386", "MEGA BUNDLE").otherwise("BUNDLE")).groupBy("date", "product_group").agg(countDistinct("account_id").alias("pu"), (sum("paid_use")/100).alias("revenue")).orderBy("date", "product_group").toPandas()

,date,product_group,pu,revenue
0,2021-11-12,BUNDLE,20807,510329.25
1,2021-11-12,MEGA BUNDLE,64834,2627735.00
2,2021-11-13,BUNDLE,8722,215884.65
3,2021-11-13,MEGA BUNDLE,19546,823725.40
4,2021-11-14,BUNDLE,5774,142335.10
5,2021-11-14,MEGA BUNDLE,11249,475714.50
6,2021-11-15,BUNDLE,4201,103091.45
7,2021-11-15,MEGA BUNDLE,8776,370286.35
8,2021-11-16,BUNDLE,3355,82044.30
9,2021-11-16,MEGA BUNDLE,6430,270158.40


In [28]:
bp_user = bp_user.groupBy("date").agg(countDistinct("accountid").alias("au")).orderBy("date").toPandas()

In [29]:
bp_user

,date,au
0,2021-08-04,1261340
1,2021-08-05,1968652
2,2021-08-06,2446718
3,2021-08-07,2899098
4,2021-08-08,3253513
5,2021-08-09,3498561
6,2021-08-10,4068393


In [12]:
bp_product_daily = bp_gcoin.groupBy("date", "product_name").agg(count("*").alias("unit_sold"), countDistinct("account_id").alias("pu"), (sum("price")/100).alias("revenue")).toPandas()

In [13]:
bp_product_daily.head()

,date,product_name,unit_sold,pu,revenue
0,2021-08-05,JENNIE'S BLACKPINK BUNDLE,5036,5036,140000.8
1,2021-08-09,JISOO'S BLACKPINK BUNDLE,1479,1479,41116.2
2,2021-08-09,LISA'S BLACKPINK BUNDLE,1143,1143,31775.4
3,2021-08-08,BLACKPINK MEGA BUNDLE,3496,3496,346104.0
4,2021-08-04,JISOO'S BLACKPINK BUNDLE,5804,5804,161351.2


In [30]:
bp_user["date"] = bp_user["date"].astype(str)
bp_product_daily["date"] = bp_product_daily["date"].astype(str)
bp_daily = pd.merge(bp_user, bp_product_daily, on='date')

,date,au,product_name,unit_sold,pu,revenue
0,2021-08-04,1261340,JISOO'S BLACKPINK BUNDLE,5804,5804,161351.2
1,2021-08-04,1261340,BLACKPINK MEGA BUNDLE,22570,22570,2234430.0
2,2021-08-04,1261340,LISA'S BLACKPINK BUNDLE,6205,6205,172499.0
3,2021-08-04,1261340,JENNIE'S BLACKPINK BUNDLE,6655,6655,185009.0
4,2021-08-04,1261340,ROSÉ'S BLACKPINK BUNDLE,8131,8131,226041.8
5,2021-08-05,1968652,JENNIE'S BLACKPINK BUNDLE,5036,5036,140000.8
6,2021-08-05,1968652,JISOO'S BLACKPINK BUNDLE,4229,4229,117566.2
7,2021-08-05,1968652,ROSÉ'S BLACKPINK BUNDLE,5439,5439,151204.2
8,2021-08-05,1968652,BLACKPINK MEGA BUNDLE,11095,11095,1098405.0
9,2021-08-05,1968652,LISA'S BLACKPINK BUNDLE,3693,3693,102665.4


In [16]:
bp_mega_buyer_list = bp_gcoin.where(col("product_name") == "BLACKPINK MEGA BUNDLE").select(col("account_id")).distinct().toPandas()["account_id"].tolist()

In [17]:
bp_bundle_buyer_list = bp_gcoin.where(col("product_name") != "BLACKPINK MEGA BUNDLE").select(col("account_id")).distinct().toPandas()["account_id"].tolist()

In [18]:
len(bp_mega_buyer_list)

53493

In [19]:
len(bp_bundle_buyer_list)

75088

In [20]:
bp_gcoin.where(col("product_name") != "BLACKPINK MEGA BUNDLE").select(count("*")).show()

+--------+
|count(1)|
+--------+
|   88706|
+--------+



In [21]:
both = []
for i in bp_bundle_buyer_list:
    if i in bp_mega_buyer_list:
        both.append(i)

In [22]:
len(both)

40

In [40]:
bp_gcoin.where(col("account_id").isin(both)).groupBy("account_id").agg(countDistinct("product_name")).toPandas()

,account_id,count(DISTINCT product_name)
0,account.2a1a88c4f04d4eeb8c2761d27ff72eca,2
1,account.0b4029542281404ebbe974beeb8c947f,2
2,account.2d28f1fb893840c5b117420c1d4cb76a,2
3,account.2f11a8e8b105402690521d60002bcb92,2
4,account.94118523cbda4768ad6df800350b9dc2,2
5,account.7efd8efa390b4b6a9b2b3df839fc97e0,2
6,account.533687c983d148148ab8302ed0460559,2
7,account.203bacd3d01b404a918fb18cf62871d0,2
8,account.93c44711f9f741bdbda1bcb15456b0bc,3
9,account.2b2127abe7ee4b8cb0c4f84aa52393c2,2


In [24]:
len(both)/len(bp_mega_buyer_list) * 100

0.8991830706821453

In [147]:
pcs5_gcoin = load_data_mart(start_date=pcs5_start_date, end_date=pcs5_end_date, device=device, table_name="gcoin_use").where(col("event_name") == pcs5_event_name)

In [33]:
pcs5_gcoin.select(col("product_name")).distinct().show()

+--------------------+
|        product_name|
+--------------------+
|       PCS5 GEAR BOX|
|PCS5 SHRAPNEL SCR...|
|PCS5 ULTIMATE BUNDLE|
|     PCS5 POWER PACK|
| PCS5 SQUAD WIPE SET|
|PCS5 SHRAPNEL SCR...|
|PCS5 PARACHUTE & ...|
+--------------------+



In [149]:
pcs5_gcoin = pcs5_gcoin.withColumn("product_group", when(col("product_id") == "itemdesc.13000328", "ULTIMATE BUNDLE").when(col("product_id").isin(["itemdesc.13000329"]), "SET").otherwise("OTHERS"))

In [152]:
pcs5_gcoin.select("product_group").distinct().show()

+---------------+
|  product_group|
+---------------+
|         OTHERS|
|            SET|
|ULTIMATE BUNDLE|
+---------------+



In [153]:
pcs5_mega_buyer_list = pcs5_gcoin.where(col("product_group") == "ULTIMATE BUNDLE").select(col("account_id")).distinct().toPandas()["account_id"].tolist()

In [155]:
pcs5_set_buyer_list = pcs5_gcoin.where(col("product_group") == "SET").select(col("account_id")).distinct().toPandas()["account_id"].tolist()

In [156]:
pcs5_other_buyer_list = pcs5_gcoin.where(col("product_group") == "OTHERS").select(col("account_id")).distinct().toPandas()["account_id"].tolist()

In [157]:
len(pcs5_mega_buyer_list)

21522

In [158]:
len(pcs5_set_buyer_list)

35490

In [159]:
len(pcs5_other_buyer_list)

62933

In [161]:
pcs5_mega_set = []
for i in pcs5_set_buyer_list:
    if i in pcs5_mega_buyer_list:
        pcs5_mega_set.append(i)

In [162]:
len(pcs5_mega_set)

189

In [164]:
pcs5_mega_other = [i for i in pcs5_mega_buyer_list if i in pcs5_other_buyer_list ]
len(pcs5_mega_other)

258

In [165]:
tg_gcoin = load_data_mart(start_date=tg_start_date, end_date=tg_end_date, device=device, table_name="gcoin_use").where(col("event_name") == tg_event_name)

In [166]:
bundle_list = ["itemdesc.13000354", "itemdesc.13000350", "itemdesc.13000349", "itemdesc.13000351", "itemdesc.13000352", "itemdesc.13000353"]
tg_gcoin = tg_gcoin.withColumn("product_group", when(col("product_id") == "itemdesc.13000348", "ULTIMATE BUNDLE").when(col("product_id").isin(bundle_list), "BUNDLE").otherwise("OTHERS"))

In [167]:
tg_gcoin.select("product_group").distinct().show()

+---------------+
|  product_group|
+---------------+
|         OTHERS|
|         BUNDLE|
|ULTIMATE BUNDLE|
+---------------+



In [49]:
tg_gcoin.select(countDistinct("product_name")).show()

+----------------------------+
|count(DISTINCT product_name)|
+----------------------------+
|                          25|
+----------------------------+



In [52]:
tg_gcoin.select("product_name", "price").distinct().orderBy("price", ascending=False).show(truncate=False)

+------------------------------------+-----+
|product_name                        |price|
+------------------------------------+-----+
|SPACE BUNNY ULTIMATE BUNDLE         |5440 |
|BUNNY AERONAUTICS WEAPON SET        |2000 |
|MOON RABBIT BUNDLE                  |1590 |
|B.A.S.A. BUNNY BUNDLE               |1590 |
|MOON SEEKER BUNDLE                  |1190 |
|Moon Rabbit Outfit                  |1190 |
|B.A.S.A. Bunny Outfit               |1190 |
|BUNNY AERONAUTICS HELMET SET        |1000 |
|BUNNY AERONAUTICS BACKPACK SET      |1000 |
|White Rabbit - Pan                  |990  |
|Bunny Aeronautics - M416            |990  |
|B.A.S.A. Bunny Jacket               |990  |
|Bunny Aeronautics - Mini14          |990  |
|Moon Rabbit Jacket                  |990  |
|Moon Seeker Costume                 |800  |
|Bunny Aeronautics Backpack (Level 3)|500  |
|Bunny Aeronautics - Helmet (Level 3)|500  |
|Bunny Aeronautics Backpack (Level 1)|500  |
|Bunny Aeronautics - Helmet (Level 1)|500  |
|Bunny Aer

In [170]:
tg_mg_buyer_list = tg_gcoin.where(col("product_group") == "ULTIMATE BUNDLE").select("account_id").distinct().toPandas()["account_id"].tolist()

In [169]:
tg_bundle_buyer_list = tg_gcoin.where(col("product_group") == "BUNDLE").select("account_id").distinct().toPandas()["account_id"].tolist()

In [171]:
tg_other_buyer_list = tg_gcoin.where(col("product_group") == "OTHERS").select("account_id").distinct().toPandas()["account_id"].tolist()

In [172]:
len(tg_mg_buyer_list)

25337

In [173]:
len(tg_bundle_buyer_list)

76938

In [174]:
len(tg_other_buyer_list)

63452

In [175]:
mg_bundle_both = []
for i in tg_bundle_buyer_list:
    if i in tg_mg_buyer_list:
        mg_bundle_both.append(i)

print(len(mg_bundle_both))

303


In [176]:
mg_other_both = []
for i in tg_other_buyer_list:
    if i in tg_mg_buyer_list:
        mg_other_both.append(i)

print(len(mg_other_both))

228


In [177]:
hw_gcoin = load_data_mart(start_date=hw_start_date, end_date=hw_end_date, device="pc", table_name="gcoin_use").where(col("event_name") == hw_event_name)

In [178]:
hw_bundle_list = ["itemdesc.13000358", "itemdesc.13000360", "itemdesc.13000357", "itemdesc.13000359", "itemdesc.13000361"]
hw_gcoin = hw_gcoin.withColumn("product_group", when(col("product_id") == "itemdesc.13000362", "ULTIMATE BUNDLE").when(col("product_id").isin(hw_bundle_list), "BUNDLE").otherwise("OTHERS"))

In [181]:
hw_gcoin.select("product_group").distinct().show()

+---------------+
|  product_group|
+---------------+
|         OTHERS|
|         BUNDLE|
|ULTIMATE BUNDLE|
+---------------+



In [182]:
hw_mg_buyer_list = hw_gcoin.where(col("product_group") == "ULTIMATE BUNDLE").select("account_id").distinct().toPandas()["account_id"].tolist()

In [184]:
hw_bundle_buyer_list = hw_gcoin.where(col("product_group") == "BUNDLE").select("account_id").distinct().toPandas()["account_id"].tolist()

In [185]:
hw_other_buyer_list = hw_gcoin.where(col("product_group") == "OTHERS").select("account_id").distinct().toPandas()["account_id"].tolist()

In [186]:
len(hw_mg_buyer_list)

2355

In [187]:
len(hw_bundle_buyer_list)

9213

In [188]:
len(hw_other_buyer_list)

25356

In [189]:
mg_bundle_both = []
for i in hw_bundle_buyer_list:
    if i in hw_mg_buyer_list:
        mg_bundle_both.append(i)
        
print(len(mg_bundle_both))

74


In [190]:
mg_other_both = []
for i in hw_other_buyer_list:
    if i in hw_mg_buyer_list:
        mg_other_both.append(i)

print(len(mg_other_both))

1219


In [192]:
hw_gcoin.where(col("product_group") == "OTHERS").where(col("account_id").isin(mg_other_both)).groupBy("product_name").agg(countDistinct("account_id").alias("pu")).orderBy("pu", ascending=False).toPandas()

,product_name,pu
0,Victory Dance 70,995
1,Dear Air,478
2,Halloween Jackpot,377
3,Molten Mayhem - M249,9
4,Night Slayer - Crossbow,5
5,Lycan Hunter - S686,3


In [87]:
pgc_gcoin = load_data_mart(start_date=pgc_start_date, end_date=pgc_end_date, device=device, table_name="gcoin_use")

no data exists in
s3a://pubg-log-labs/data_mart/economy_v3/gcoin_use/pc/2021-11-18


In [88]:
pgc_gcoin = pgc_gcoin.where(col("event_name") == pgc_event_name)

In [90]:
pgc_gcoin.select("product_name", "product_id", "price").distinct().orderBy("price", ascending=False).show(truncate=False)

+--------------------------------------------------------+-----------------+-----+
|product_name                                            |product_id       |price|
+--------------------------------------------------------+-----------------+-----+
|PGC 2021 MEGA BUNDLE + VOTING COUPONS                   |itemdesc.13000386|5940 |
|PGC 2021 CONTINGENT CHAOS JACKET BUNDLE + VOTING COUPONS|itemdesc.13000384|3170 |
|PGC 2021 CONTINGENT CHAOS DUSTER BUNDLE + VOTING COUPONS|itemdesc.13000383|3170 |
+--------------------------------------------------------+-----------------+-----+



In [92]:
pgc_mg_buyer_list = pgc_gcoin.where(col("product_id") == "itemdesc.13000386").select("account_id").distinct().toPandas()["account_id"].tolist()

In [93]:
pgc_bundle_buyer_list = pgc_gcoin.where(col("product_id").isin(["itemdesc.13000384", "itemdesc.13000383"])).select("account_id").distinct().toPandas()["account_id"].tolist()

In [94]:
len(pgc_mg_buyer_list)

116162

In [95]:
len(pgc_bundle_buyer_list)

45000

In [96]:
both = []
for i in pgc_bundle_buyer_list:
    if i in pgc_mg_buyer_list:
        both.append(i)
    
print(len(both))

427
